In [110]:
import pandas as pd

In [111]:
# Carregar dataset
df = pd.read_csv('../datasets/dataset-case_study.csv')

# Informações sobre o conjunto de dados
print(df.shape)
print(df.columns)

# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)

# Contar quantas linhas tem para cada plataforma
year_counts = df['year'].value_counts()
print("\nDistribuição por ano:", year_counts)

(5933, 9)
Index(['restaurante_name', 'user_name', 'review_date', 'stars', 'comment_text',
       'year', 'platform', 'language', 'comment_text_translated'],
      dtype='object')

Distribuição por plataforma: platform
Google             3917
Facebook            847
Foursquare          834
Yelp                310
Zomato               24
Restaurant Guru       1
Name: count, dtype: int64

Distribuição por ano: year
2023       1247
2018        796
2017        667
2019        645
2022        515
2024        506
2020        475
2016        358
2015        325
2021        256
Unknown     126
2014         17
Name: count, dtype: int64


In [112]:
# Remover ocorrências de erros
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Comida: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Comida: 4', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Serviço: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Serviço: 4', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Atmosfera: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('Atmosfera: 4', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('comida: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('comida: 4', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('serviço: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('serviço: 4', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('atmosfera: 5', '')
df['comment_text_translated'] = df['comment_text_translated'].str.replace('atmosfera: 4', '')

# Realizar strip
df['comment_text_translated'] = df['comment_text_translated'].astype(str).apply(lambda x: x.strip())

# Remover sentenças nulas
print("\nLinhas com valores nulos:\n", df['comment_text_translated'].isnull().sum())
print('Removendo comentários nulos...')
df = df.dropna(subset=['comment_text_translated']).reset_index(drop=True)

# Remover sentenças duplicadas
comentarios_duplicados = df.duplicated(subset="comment_text_translated").sum()
print("\nTotal de comentários duplicados:", comentarios_duplicados)
print('Removendo comentários duplicados...\n')
df = df.drop_duplicates(subset="comment_text_translated", keep='first').reset_index(drop=True)


Linhas com valores nulos:
 0
Removendo comentários nulos...

Total de comentários duplicados: 940
Removendo comentários duplicados...



In [113]:
# Informações sobre o conjunto de dados
print(df.shape)

# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)

# Contar quantas linhas tem para cada plataforma
year_counts = df['year'].value_counts()
print("\nDistribuição por ano:", year_counts)

(4993, 9)

Distribuição por plataforma: platform
Google             3243
Foursquare          731
Facebook            711
Yelp                285
Zomato               22
Restaurant Guru       1
Name: count, dtype: int64

Distribuição por ano: year
2023       960
2018       699
2017       596
2019       563
2022       492
2020       439
2016       321
2024       316
2015       267
2021       239
Unknown     85
2014        16
Name: count, dtype: int64


In [114]:
df.to_csv(f'../datasets/dataset-case_study-tratado.csv', index=False)